## Check your connection to Generative AI Hub


👉 First you need to assign the values from `generative-ai-codejam/.aicore-config.json` to the environmental variables. 

That way the Generative AI Hub [Python SDK](https://pypi.org/project/generative-ai-hub-sdk/) will connect to Generative AI Hub.

👉 For the Python SDK to know which resource group to use, you also need to set the `resource group` in the `variables.py` file to your own `resource group` (e.g. **team-01**) that you created in the SAP AI Launchpad in exercise [00-connect-AICore-and-AILaunchpad](000-connect-AICore-and-AILaunchpad.md).

In [ ]:
import init_env
import variables

init_env.set_environment_variables()
# test whether the resource group is assigned correctly
variables.RESOURCE_GROUP

## Prompt an LLM with Generative AI Hub

👉 Open [variables.py](variables.py) and change the LLM_DEPLOYMENT_ID to your deployment ID. You find your deployment ID in the SAP AI Launchpad under the `ML Operations` tab under `Deployments`.

In [ ]:
from gen_ai_hub.proxy.native.openai import chat

messages = [
    {
        "role": "user", 
        "content": "What is the underlying model architecture of an LLM? Explain it as short as possible."
    }
]

kwargs = dict(deployment_id=variables.LLM_DEPLOYMENT_ID, messages=messages)
response = chat.completions.create(**kwargs)
print(response.choices[0].message.content)

## Understanding roles
Most LLMs have the roles `system`, `assistant` (GPT) or `model` (Gemini) and `user` that can be used to steer the models response. In the previous step you only used the role `user` to ask your question. 

👉 Try out different `system` messages to change the response. You can also tell the model to not engage in smalltalk or only answer questions on a certain topic. Then try different user prompts as well!

In [ ]:
messages = [
    {   "role": "system", 
        "content": "Speak like Yoda from Star Wars."
    }, 
    {
        "role": "user", 
        "content": "What is the underlying model architecture of an LLM? Explain it as short as possible."
    }
]

kwargs = dict(deployment_id=variables.LLM_DEPLOYMENT_ID, messages=messages)
response = chat.completions.create(**kwargs)
print(response.choices[0].message.content)

👉 Also try to have it speak like a pirate.

👉 Now let's be more serious! Tell it to behave like an SAP consultant talking to AI Developers.

👉 Ask it to behave like an SAP Consultant talking to ABAP Developers and to make ABAP comparisons.

## Hallucinations
👉 Run the following question.

In [ ]:
messages = [
    {   "role": "system", 
        "content": "You are an SAP Consultant."
    }, 
    {
        "role": "user", 
        "content": "How does the data masking of the orchestration service work?"
    }
]

kwargs = dict(deployment_id=variables.LLM_DEPLOYMENT_ID, messages=messages)
response = chat.completions.create(**kwargs)
print(response.choices[0].message.content)

☝️ Compare the response to [SAP Help - Generative AI Hub SDK](https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/orchestration-service.html). 

👉 What did the model respond? Was it the truth or a hallucination?

👉 Which questions work well, which questions still do not work so well?

[Next exercise](04-create-embeddings.ipynb)